In [1]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.functions import count, col, sum



In [2]:
spark = SparkSession.builder \
    .appName("spark-hw") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
print(spark.version)

3.5.1


In [4]:
actor_df = spark.read.csv('../data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('../data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('../data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('../data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('../data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('../data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('../data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('../data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('../data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('../data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('../data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('../data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('../data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('../data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('../data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
from pyspark.sql.functions import col

film_category_count_df = (
    film_category_df
    .join(category_df, film_category_df.category_id == category_df.category_id, "inner")
    .groupBy("name")
    .count()
    .orderBy(col("count").desc())
)

film_category_count_df.show()


+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [6]:
top_actors_df = (
    rental_df
    .join(inventory_df, "inventory_id")
    .join(film_actor_df, "film_id")
    .join(actor_df, "actor_id")
    .groupBy("first_name", "last_name")
    .agg(count("*").alias("rental_count"))
    .orderBy(col("rental_count").desc())
    .limit(10)
)

top_actors_df.show()

+----------+-----------+------------+
|first_name|  last_name|rental_count|
+----------+-----------+------------+
|     SUSAN|      DAVIS|         825|
|      GINA|  DEGENERES|         753|
|   MATTHEW|     CARREY|         678|
|      MARY|     KEITEL|         674|
|    ANGELA|WITHERSPOON|         654|
|    WALTER|       TORN|         640|
|     HENRY|      BERRY|         612|
|     JAYNE|      NOLTE|         611|
|       VAL|     BOLGER|         605|
|    SANDRA|     KILMER|         604|
+----------+-----------+------------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [7]:
category_revenue_df = (
    payment_df
    .join(rental_df, "rental_id")
    .join(inventory_df, "inventory_id")
    .join(film_df, "film_id")
    .join(film_category_df, "film_id")
    .join(category_df, "category_id")
    .groupBy("name")
    .agg(sum("amount").alias("total_revenue"))
    .orderBy(col("total_revenue").desc())
    .limit(1)
)

category_revenue_df.show()

+------+-----------------+
|  name|    total_revenue|
+------+-----------------+
|Sports|5314.209999999848|
+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [8]:
films_not_in_inventory_df = (
    film_df
    .join(inventory_df, on="film_id", how="left_anti")
    .select("title")
)

films_not_in_inventory_df.show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [9]:
top_children_actors_df = (
    film_actor_df
    .join(film_category_df, "film_id")
    .join(category_df, "category_id")
    .filter(col("name") == "Children")
    .join(actor_df, "actor_id")
    .groupBy("first_name", "last_name")
    .agg(count("*").alias("appearance_count"))
    .orderBy(col("appearance_count").desc())
    .limit(3)
)

top_children_actors_df.show()


+----------+---------+----------------+
|first_name|last_name|appearance_count|
+----------+---------+----------------+
|     HELEN|   VOIGHT|               7|
|     SUSAN|    DAVIS|               6|
|     RALPH|     CRUZ|               5|
+----------+---------+----------------+



Stop Spark session:

In [10]:
spark.stop()